# Potato Disease Detection using CNN

## 📘 Overview

This project uses Convolutional Neural Networks (CNNs) to classify potato leaf images into three categories:

Early Blight

Late Blight

Healthy

We’ll use the Potato Disease Detection dataset from Kaggle, preprocess it, train a CNN model, and evaluate its accuracy.

## 1. Import Required Libraries

In [ ]:
import kagglehub
import os
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory


## 2. Load Dataset from Kaggle

In [ ]:
# Download latest version
path = kagglehub.dataset_download("mgmitesh/potato-disease-detection-dataset")

print("Path to dataset files:", path)

## 3. Create Training and Validation Datasets



In [ ]:
train_ds=image_dataset_from_directory(
    directory=path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256,256),
    batch_size=32
)
test_ds=image_dataset_from_directory(
    directory=path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256,256),
    batch_size=32
)

In [ ]:
class_names=train_ds.class_names
n_classes=len(class_names)
print(class_names)

## 4. Normalize and Optimize Datasets

In [ ]:
def resize_and_rescale(image):
  image=image/255.0
  return tf.image.resize(image,(256,256))

train_ds=train_ds.map(lambda x,y: (resize_and_rescale(x),y))
test_ds=test_ds.map(lambda x,y: (resize_and_rescale(x),y))

autotune=tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=autotune)
test_ds=test_ds.cache().prefetch(buffer_size=autotune)

# 5. Build CNN Model

In [ ]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)),
    MaxPooling2D((2,2)),
     Dropout(0.1),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
     Dropout(0.1),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.1),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
     Dropout(0.1),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
     Dropout(0.1),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.3),
    Dense(64,activation='relu'),
    Dense(n_classes,activation='softmax')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']

)


## 6. Train the Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(
    monitor='loss',
    patience=5
)
history=model.fit(train_ds,epochs=40,callbacks=[early_stopping],verbose=2)

In [ ]:
model.evaluate(test_ds,verbose=2)


## Visualize Training Performance

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Model Training Performance')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()


## 8. Evaluate on Test Dataset

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"✅ Test Accuracy: {test_acc*100:.2f}%")


## 9. Confusion Matrix & Classification Report

In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))
